In [1]:
import mysql.connector
import pandas as pd

class DBManager :
    
    def __init__(self, host, port, user, password, database):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.db = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database
        )
        self.cursor = self.db.cursor(buffered=True)
        print("DB connected")

    def close(self):
        if self.cursor:
            self.cursor.close()
        if self.db:
            self.db.close()
        print("DB disconnected")

    def select_all(self, table):
        sql = f"SELECT * FROM {table}"
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        columns = [col[0] for col in self.cursor.description]
        df = pd.DataFrame(result, columns=columns).reset_index(drop=True)
        return df

    def execute(self, sql):
        self.cursor.execute(sql)
        self.db.commit()
        print("------------------------------------------")
        print("EXECUTED")
        if self.cursor.description is not None:
            result = self.cursor.fetchall()
            print(result)
        return True
    
dbm = DBManager(
    host="jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port= 3306,
    user="root",
    password="jinhyuk2dacibul",
    database="project1"
)


DB connected


In [ ]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
from mysql.connector import connect

# 네이버 API에서 "동"과 "주소" 정보를 받아오는 함수
def get_dong_and_address(latitude, longitude, client_id, client_secret):
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
    coords = f"{longitude},{latitude}"
    output = "json"
    orders = "addr"
    
    url = f"{endpoint}?coords={coords}&output={output}&orders={orders}"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            if 'results' in data and data['results']:
                # "동" 정보와 "주소" 추출
                dong = data['results'][0]['region']['area3']['name']
                full_address = f"{data['results'][0]['region']['area1']['name']} {data['results'][0]['region']['area2']['name']} {dong}"
                return dong, full_address
            else:
                return None, None
        else:
            print(f"API 요청 실패: {response.status_code}")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"요청 중 오류 발생: {e}")
        return None, None


